
# Amazon — Regresión Lineal Múltiple (Colab-ready)

**Objetivo:** predecir `discounted_price` (precio con descuento) usando variables numéricas limpias y **categoría** como dummies.  
Incluye: limpieza, train/test, métricas (`R²`, `RMSE`), diagnósticos (residuales, **Breusch–Pagan**, **VIF**), y comparación con **Ridge/Lasso**.

> **Cómo usar**: Ejecutá las celdas en orden. Si cambiás el path o querés probar otro objetivo, modificá las variables donde se indica.


In [ ]:

# === 0) Montar Google Drive ===
from google.colab import drive
drive.mount('/content/drive', force_remount=True)


In [ ]:

# === 1) Imports e instalación opcional ===
import sys, os, re, numpy as np, pandas as pd, matplotlib.pyplot as plt

# Si hace falta, descomentá:
# !pip install statsmodels scikit-learn

import statsmodels.api as sm
import statsmodels.stats.api as sms
from statsmodels.stats.outliers_influence import variance_inflation_factor as VIF

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression, RidgeCV, LassoCV
from sklearn.metrics import r2_score, mean_squared_error
from math import sqrt
plt.rcParams['figure.figsize'] = (7,4)


In [ ]:

# === 2) Configurar ruta al CSV en tu Drive ===
# Ajustá la ruta a tu carpeta real. Recomendado usar 'MyDrive' sin espacios en Colab.
BASE = '/content/drive/MyDrive'
CSV_PATH = f'{BASE}/Dataset/amazon.csv'  # <-- CAMBIÁ si está en otra carpeta

print('Existe el archivo?', os.path.exists(CSV_PATH))
if not os.path.exists(CSV_PATH):
    print('⚠️ Revisá la ruta CSV_PATH arriba.')


In [ ]:

# === 3) Lectura y limpieza mínima ===
df_raw = pd.read_csv(CSV_PATH, encoding='utf-8')

def to_num(series: pd.Series) -> pd.Series:
    s = series.astype(str).str.replace(r'[^0-9\.]', '', regex=True)
    # Si hubiera más de un punto decimal, conservamos el primero
    s = s.apply(lambda x: x if x.count('.') <= 1 else x.replace('.', '', x.count('.')-1))
    return pd.to_numeric(s, errors='coerce')

df = df_raw.copy()
for col in ['discounted_price','actual_price','discount_percentage','rating','rating_count']:
    if col in df.columns:
        df[col + '_num'] = to_num(df[col])

keep_cols = ['discounted_price_num','actual_price_num','discount_percentage_num','rating_num','rating_count_num','category']
data = df[keep_cols].dropna()

print('Shape luego de limpieza:', data.shape)
data.head()


In [ ]:

# === 4) Features y objetivo ===
y = data['discounted_price_num']

# Dummies de categoría (drop_first para evitar multicolinealidad perfecta)
X_num = data[['actual_price_num','discount_percentage_num','rating_num','rating_count_num']].copy()
X_cat = pd.get_dummies(data['category'], prefix='cat', drop_first=True)
X = pd.concat([X_num, X_cat], axis=1)

# Train/Test split
Xtr, Xte, ytr, yte = train_test_split(X, y, test_size=0.2, random_state=42)
Xtr.shape, Xte.shape


In [ ]:

# === 5) Regresión Lineal Múltiple (baseline) ===
lin = LinearRegression().fit(Xtr, ytr)
pred = lin.predict(Xte)

r2 = r2_score(yte, pred)
rmse = sqrt(mean_squared_error(yte, pred))

print(f'R² = {r2:.4f} | RMSE = {rmse:.2f}')
print('N features:', X.shape[1])

# Scatter pred vs real (línea ideal)
plt.figure()
plt.scatter(yte, pred, s=10)
mn, mx = min(yte.min(), pred.min()), max(yte.max(), pred.max())
plt.plot([mn, mx], [mn, mx], '--')
plt.xlabel('y real (test)'); plt.ylabel('y predicho')
plt.title('Predicho vs Real (ideal = línea punteada)')
plt.show()


In [ ]:

# === 6) Diagnósticos: residuos y Breusch–Pagan ===
res = yte - pred

plt.figure()
plt.axhline(0, linestyle='--')
plt.scatter(pred, res, s=10)
plt.xlabel('Predicción'); plt.ylabel('Residual')
plt.title('Residuos vs Predicción (buscar nube sin forma)')
plt.show()

# Breusch–Pagan (usamos OLS con constante sobre todo el set limpio para estabilidad)
X_const = sm.add_constant(X, has_constant='add')
ols = sm.OLS(y, X_const).fit()
bp_stat, bp_pval, f_stat, f_pval = sms.het_breuschpagan(ols.resid, X_const)

print('Breusch–Pagan:')
print('  LM =', bp_stat, '| p-valor =', bp_pval)
print('  F  =', f_stat,  '| p-valor =', f_pval)

# Si hay heteroscedasticidad, ejemplo de ajuste con errores robustos (HC3)
ols_hc3 = sm.OLS(y, X_const).fit(cov_type='HC3')
print('\nOLS con errores robustos (HC3) — R²:', round(ols_hc3.rsquared, 4))


In [ ]:

# === 7) VIF (multicolinealidad) — solo en las X numéricas principales para interpretabilidad ===
Xv = X_num.copy()  # enfocar VIF en numéricas base
Xv = Xv.assign(const=1.0)  # agregar constante al final si hiciera falta (no se calcula VIF para 'const')
cols = Xv.columns[:-1]     # sin la constante

vif_rows = []
for i, c in enumerate(cols):
    val = VIF(Xv[cols].values, i)
    vif_rows.append((c, val))

vif_df = pd.DataFrame(vif_rows, columns=['feature','VIF']).sort_values('VIF', ascending=False)
vif_df


In [ ]:

# === 8) Regularización: Ridge y Lasso (comparación) ===
alphas = np.logspace(-3, 3, 25)

ridge = RidgeCV(alphas=alphas, cv=5).fit(Xtr, ytr)
pred_r = ridge.predict(Xte)
print(f'Ridge  α*={ridge.alpha_:.4f} | R² = {r2_score(yte, pred_r):.4f} | RMSE = {sqrt(mean_squared_error(yte, pred_r)):.2f}')

lasso = LassoCV(cv=5, n_alphas=50, random_state=42, max_iter=5000).fit(Xtr, ytr)
pred_l = lasso.predict(Xte)
print(f'Lasso  α*={lasso.alpha_:.6f} | R² = {r2_score(yte, pred_l):.4f} | RMSE = {sqrt(mean_squared_error(yte, pred_l)):.2f}')
print('Lasso — # coeficientes ≠ 0:', np.sum(lasso.coef_ != 0))


In [ ]:

# === 9) (Opcional) Transformación log del objetivo si hay heteroscedasticidad fuerte ===
y_log = np.log1p(y)
Xtr_l, Xte_l, ytr_l, yte_l = train_test_split(X, y_log, test_size=0.2, random_state=42)

lin_l = LinearRegression().fit(Xtr_l, ytr_l)
pred_lg = lin_l.predict(Xte_l)

r2_lg = r2_score(yte_l, pred_lg)
rmse_lg = sqrt(mean_squared_error(yte_l, pred_lg))
print(f'[LOG y] R² = {r2_lg:.4f} | RMSE (escala log) = {rmse_lg:.4f}')

# Volver a la escala original para interpretar el error (aprox)
pred_back = np.expm1(pred_lg)
yte_back  = np.expm1(yte_l)
print('RMSE (escala original, aprox):', round(sqrt(mean_squared_error(yte_back, pred_back)), 2))
